In [1]:
#!pip install transformers
#!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import os
import re
import glob

import torch.nn as nn
import xml.etree.ElementTree as ET
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import random
from os import listdir
from os.path import isfile, join
from sklearn import preprocessing
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import Trainer, TrainingArguments
from datasets import load_metric
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
import ast
# from argparse import ArgumentParser
# parser = ArgumentParser()
# parser.add_argument("-multi_lb_classi", default=1, type=int, help="whether train a multi-lable classifier or individual classifiers for each class")
# opt = parser.parse_args()
multi_lb_classi = 0
docs = os.listdir("./scisumm-2018/Training")
train_e = int(0.8*len(docs))
train_docs = docs[:train_e]
test_docs = docs[train_e:]
# test_docs  = os.listdir("./scisumm-2018/Test")
stop_words = set(stopwords.words('english'))

def preprocess(example_sent):
    global stop_words
    word_tokens = word_tokenize(example_sent.lower())
    filtered_sentence = [w for w in word_tokens if not w in stop_words and w.isalpha()]
    new = " " 
    a = new.join(filtered_sentence)
    return a



2021-12-07 15:43:18.167643: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/apps/cuDNN/7.6.5-cuda-10.2/lib64
2021-12-07 15:43:18.167688: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_dataset(files, folder):
    data_lis = []
    labels = []
    for z,f in enumerate(files):
        data = None
        ann = None
        a_folder = folder+f+"/annotation/"
        file = os.listdir(a_folder)[0]

        ann = a_folder+file
        with open(ann,"r") as f:
            data = f.read()
#         print(f)
            facet = re.findall("Discourse Facet:\s+([^|]*)", data)
            for f in facet:
                if '[' in f:
                    f = ast.literal_eval(f)
                labels.append(f)

            
            ref_text = re.findall("Reference Text:\s+([^|]*)", data)
            pattern = r'\<.*?\>'
            pattern2 = r'\(.*?\)'
            for ref in ref_text:
                ref = re.sub(pattern2,'',re.sub(pattern, '', ref))
                ref = preprocess(ref)
                data_lis.append(ref)

            
    return data_lis, labels


In [3]:
# def get_test_dataset(test_docs):
#     '''
#     gives citance and labels
#     '''
#     data_lis = []
#     labels = []
#     for z,f in enumerate(files):
#         df = folder+f+"/annotation/"+f+".csv"
#         data_lis.append()

# test_data,test_labels = get_dataset(test_docs[:2],folder='scisumm-2018/Test/')



In [4]:
train_data, train_labels = get_dataset(train_docs,"scisumm-2018/Training/")
test_data, test_labels = get_dataset(test_docs,"scisumm-2018/Training/")

In [5]:
train_data

['paper propose probabilistic admixture model capture latent topics underlying context pairs',
 'formalism parallel within assumed constitute mixture hidden topics follows bilingual translation paper propose probabilistic admixture model capture latent topics underlying context pairs',
 'propose new statistical formalism bilingual topic admixture model bitam facilitate word alignment translation lexicon p f',
 'translation lexicons learned',
 'two retrieval schemes designed bitams alignment alignment takes intersection two directions generates alignments',
 'translation lexicons bf e k potential size v assuming vocabulary sizes languages v data sparsity poses serious problem estimating bf e k monolingual case instance reduce data sparsity problem introduce two remedies laplace smoothing',
 'formalism parallel within assumed constitute mixture hidden topics follows bilingual translation bitam models proposed capture topic sharing different levels linguistic granularity propose new stati

In [6]:
def split_two_labels(data, labels):
    '''
    split two labels in 2 data-points
    '''
    labels_new = []
    data_new = []
    for i,lb in enumerate(labels):
        if type(lb) == list: #needs to be split
            for l in lb:
                l = l.lower()
                if '_' in l:
                    l = l.split('_')[0]
                else:
                    l = l.split()[0]
                data_new.append(data[i])
        else:
            l = lb.lower()
            if '_' in l:
                    l = l.split('_')[0]
            else:
                l = l.split()[0]
            if l=='results':
                l = 'result'
            labels_new.append(l)
            data_new.append(data[i])
    return data_new, labels_new

train_data, train_labels = split_two_labels(train_data, train_labels)
test_data, test_labels  = split_two_labels(test_data, test_labels )

In [7]:
train_labels

['method',
 'method',
 'result',
 'implication',
 'method',
 'implication',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'result',
 'result',
 'method',
 'method',
 'method',
 'result',
 'result',
 'method',
 'method',
 'method',
 'aim',
 'method',
 'result',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'method',
 'aim',
 'method',
 'aim',
 'aim',
 'method',
 'method',
 'result',
 'method',
 'result',
 'method',
 'result',
 'aim',
 'method',
 'method',
 'result',
 'result',
 'result',
 'aim',
 'aim',
 'aim',
 'result',
 'result',
 'result',
 'method',
 'aim',
 'method',
 'aim',
 'method',
 'method',
 'result',
 'result',
 'method',
 'method',
 'method',
 'implication',
 'aim',
 'implication',
 'method',
 'method',
 'method',
 'implication',
 'result',
 'method',
 'method',
 'implication',
 'aim',
 'method',
 'met

In [8]:
le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
# val_labels = le.transform(val_labels)
test_labels = le.transform(test_labels)

In [9]:
def calc_metrics(pred_lis, gt_lis):
    acc = accuracy_score(pred_lis, gt_lis)
    recall = recall_score(pred_lis, gt_lis,average='micro')
    prec = precision_score(pred_lis, gt_lis,average='micro')
    f1 = f1_score(pred_lis, gt_lis,average='micro')
    print("metrics obtained in test: accuracy {} recall {}, precision {}, f1-score {}".format(acc,recall,prec,f1))
    
def get_integer_mapping(le):
    '''
    Return a dict mapping labels to their integer values
    from an SKlearn LabelEncoder
    le = a fitted SKlearn LabelEncoder
    '''
    res = {}
    for cl in le.classes_:
        res.update({le.transform([cl])[0]:cl})

    return res


mapping = get_integer_mapping(le)
mapping

{0: 'aim', 1: 'hypothesis', 2: 'implication', 3: 'method', 4: 'result'}

In [10]:
# val_data,val_labels = get_dataset(val_docs,folder='From-ScisummNet-2019')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)
# val_encodings = tokenizer(val_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)
test_encodings = tokenizer(test_data,return_tensors="pt",max_length=512,padding='max_length',truncation=True)


class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, torch.from_numpy(train_labels))
# val_dataset = IMDbDataset(val_encodings, torch.from_numpy(val_labels))
test_dataset = IMDbDataset(test_encodings, torch.from_numpy(test_labels))


import os
os.environ['CUDA_LAUNCH_BLOCKING']="1"


In [11]:



multi_lb_classi = True
if multi_lb_classi:

    training_args = TrainingArguments(
        output_dir='/ssd_scratch/cvit/results',          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=1,  # batch size per device during training
        per_device_eval_batch_size=1,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='/ssd_scratch/cvit/logs',            # directory for storing logs
        logging_steps=200000,
        save_steps = 30000
    )

    # model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(mapping))
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    #model = nn.DataParallel(model,device_ids=[0,1,2,3])
    

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset          # training dataset
        # eval_dataset=val_dataset             # evaluation dataset
    )

    trainer.train()
    model.eval()
    # trainer.evaluate()
    
    torch.save(model,"task1b_bert.pth")
    # val_dataloader = DataLoader(val_dataset, batch_size=8)
    test_dataloader = DataLoader(test_dataset, batch_size=8)
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    pred_lis = []
    gt_lis = []
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        pred_lis.extend(predictions.cpu().numpy())
        gt_lis.extend(batch["labels"].cpu().numpy())
        

    calc_metrics(pred_lis, gt_lis)


    gt_lis = np.array(gt_lis)
    pred_lis = np.array(pred_lis)


    #class-wise metrics
    for cl in mapping:
        print("for class: ",mapping[cl])
        ind = np.where(gt_lis==cl)
        small_pred = pred_lis[ind]
        calc_metrics(small_pred, gt_lis[ind])

# ## Classifiers for each class
else:
    # train_data = np.array(train_data)
    # test_data = np.array(test_data)

    # train_encodings = tokenizer(list(train_data),return_tensors="pt",max_length=512,padding='max_length',truncation=True)
    # test_encodings = tokenizer(list(test_data),return_tensors="pt",max_length=512,padding='max_length',truncation=True)
    print("starting")
    for cl in mapping:
#     cl = 2
        training_args = TrainingArguments(
        output_dir='/ssd_scratch/cvit/avani.gupta/results'+str(cl),          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=1,  # batch size per device during training
        per_device_eval_batch_size=1,   # batch size for evaluation
        warmup_steps=500,               # number of warmup steps for learning rate scheduler
        weight_decay=0.01,              # strength of weight decay
        logging_dir='/ssd_scratch/cvit/avani.gupta/logs'+str(cl),            # directory for storing logs
        logging_steps=20000,
        save_steps = 30000
        )
        print("for class: ",mapping[cl])
        train_ind = np.where(train_labels==cl)[0].astype(int)
        new_train_labels = np.zeros(train_labels.shape).astype(int)
        new_train_labels[train_ind] = 1

        # val_ind = np.where(val_labels==cl)[0].astype(int)
        test_ind = np.where(test_labels==cl)[0].astype(int)
        new_test_labels = np.zeros(test_labels.shape).astype(int)
        new_test_labels[test_ind] = 1


        train_dataset = IMDbDataset(train_encodings, torch.from_numpy(new_train_labels))
        test_dataset = IMDbDataset(test_encodings, torch.from_numpy(new_test_labels))    


        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        model.to(device)
        #model = nn.DataParallel(model,device_ids=[0,1,2,3])

        trainer = Trainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            args=training_args,                  # training arguments, defined above
            train_dataset=train_dataset         # training dataset
        )
        trainer.train()
        # eval
        model.eval()
        torch.save(model, "bert_single_lb_classi"+str(cl)+".pth")
        val_dataloader = DataLoader(test_dataset, batch_size=8)
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        pred_lis = []
        gt_lis = []

        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            pred_lis.extend(predictions.cpu().numpy())
            gt_lis.extend(batch["labels"].cpu().numpy())

        calc_metrics(pred_lis, gt_lis)





Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




metrics obtained in test: accuracy 0.75 recall 0.75, precision 0.75, f1-score 0.75
for class:  aim
metrics obtained in test: accuracy 0.0 recall 0.0, precision 0.0, f1-score 0.0
for class:  hypothesis
metrics obtained in test: accuracy 0.0 recall 0.0, precision 0.0, f1-score 0.0
for class:  implication
metrics obtained in test: accuracy 0.0 recall 0.0, precision 0.0, f1-score 0.0
for class:  method
metrics obtained in test: accuracy 1.0 recall 1.0, precision 1.0, f1-score 1.0
for class:  result
metrics obtained in test: accuracy 0.0 recall 0.0, precision 0.0, f1-score 0.0


In [12]:
multi_lb_classi = False
if multi_lb_classi:

    training_args = TrainingArguments(
        output_dir='/ssd_scratch/cvit/results',          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=1,  # batch size per device during training
        per_device_eval_batch_size=1,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='/ssd_scratch/cvit/logs',            # directory for storing logs
        logging_steps=200000,
        save_steps = 30000
    )

    # model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(mapping))
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    #model = nn.DataParallel(model,device_ids=[0,1,2,3])
    

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset          # training dataset
        # eval_dataset=val_dataset             # evaluation dataset
    )

    trainer.train()
    model.eval()
    # trainer.evaluate()
    
    torch.save(model,"task1b_bert.pth")
    # val_dataloader = DataLoader(val_dataset, batch_size=8)
    test_dataloader = DataLoader(test_dataset, batch_size=8)
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    pred_lis = []
    gt_lis = []
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        pred_lis.extend(predictions.cpu().numpy())
        gt_lis.extend(batch["labels"].cpu().numpy())
        

    calc_metrics(pred_lis, gt_lis)


    gt_lis = np.array(gt_lis)
    pred_lis = np.array(pred_lis)


    #class-wise metrics
    for cl in mapping:
        print("for class: ",mapping[cl])
        ind = np.where(gt_lis==cl)
        small_pred = pred_lis[ind]
        calc_metrics(small_pred, gt_lis[ind])

# ## Classifiers for each class
else:
    # train_data = np.array(train_data)
    # test_data = np.array(test_data)

    # train_encodings = tokenizer(list(train_data),return_tensors="pt",max_length=512,padding='max_length',truncation=True)
    # test_encodings = tokenizer(list(test_data),return_tensors="pt",max_length=512,padding='max_length',truncation=True)
    print("starting")
    for cl in mapping:
#     cl = 2
        training_args = TrainingArguments(
        output_dir='/ssd_scratch/cvit/avani.gupta/results'+str(cl),          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=1,  # batch size per device during training
        per_device_eval_batch_size=1,   # batch size for evaluation
        warmup_steps=500,               # number of warmup steps for learning rate scheduler
        weight_decay=0.01,              # strength of weight decay
        logging_dir='/ssd_scratch/cvit/avani.gupta/logs'+str(cl),            # directory for storing logs
        logging_steps=20000,
        save_steps = 30000
        )
        print("for class: ",mapping[cl])
        train_ind = np.where(train_labels==cl)[0].astype(int)
        new_train_labels = np.zeros(train_labels.shape).astype(int)
        new_train_labels[train_ind] = 1

        # val_ind = np.where(val_labels==cl)[0].astype(int)
        test_ind = np.where(test_labels==cl)[0].astype(int)
        new_test_labels = np.zeros(test_labels.shape).astype(int)
        new_test_labels[test_ind] = 1


        train_dataset = IMDbDataset(train_encodings, torch.from_numpy(new_train_labels))
        test_dataset = IMDbDataset(test_encodings, torch.from_numpy(new_test_labels))    


        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        model.to(device)
        #model = nn.DataParallel(model,device_ids=[0,1,2,3])

        trainer = Trainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            args=training_args,                  # training arguments, defined above
            train_dataset=train_dataset         # training dataset
        )
        trainer.train()
        # eval
        model.eval()
        torch.save(model, "bert_single_lb_classi"+str(cl)+".pth")
        val_dataloader = DataLoader(test_dataset, batch_size=8)
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        pred_lis = []
        gt_lis = []

        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            pred_lis.extend(predictions.cpu().numpy())
            gt_lis.extend(batch["labels"].cpu().numpy())

        calc_metrics(pred_lis, gt_lis)





PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


starting
for class:  aim


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/avani.gupta/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


...

metrics obtained in test: accuracy 0.75 recall 0.75, precision 0.75, f1-score 0.75
for class:  result


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/avani.gupta/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




metrics obtained in test: accuracy 0.8974358974358975 recall 0.8974358974358975, precision 0.8974358974358975, f1-score 0.8974358974358975


In [13]:
multi_lb_classi = False
if multi_lb_classi:

    training_args = TrainingArguments(
        output_dir='/ssd_scratch/cvit/results',          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=1,  # batch size per device during training
        per_device_eval_batch_size=1,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='/ssd_scratch/cvit/logs',            # directory for storing logs
        logging_steps=200000,
        save_steps = 30000
    )

    # model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(mapping))
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    #model = nn.DataParallel(model,device_ids=[0,1,2,3])
    

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset          # training dataset
        # eval_dataset=val_dataset             # evaluation dataset
    )

    trainer.train()
    model.eval()
    # trainer.evaluate()
    
    torch.save(model,"task1b_bert.pth")
    # val_dataloader = DataLoader(val_dataset, batch_size=8)
    test_dataloader = DataLoader(test_dataset, batch_size=8)
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    pred_lis = []
    gt_lis = []
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        pred_lis.extend(predictions.cpu().numpy())
        gt_lis.extend(batch["labels"].cpu().numpy())
        

    calc_metrics(pred_lis, gt_lis)


    gt_lis = np.array(gt_lis)
    pred_lis = np.array(pred_lis)


    #class-wise metrics
    for cl in mapping:
        print("for class: ",mapping[cl])
        ind = np.where(gt_lis==cl)
        small_pred = pred_lis[ind]
        calc_metrics(small_pred, gt_lis[ind])

# ## Classifiers for each class
else:
    # train_data = np.array(train_data)
    # test_data = np.array(test_data)

    # train_encodings = tokenizer(list(train_data),return_tensors="pt",max_length=512,padding='max_length',truncation=True)
    # test_encodings = tokenizer(list(test_data),return_tensors="pt",max_length=512,padding='max_length',truncation=True)
    print("starting")
    for cl in mapping:
#     cl = 2
        training_args = TrainingArguments(
        output_dir='/ssd_scratch/cvit/avani.gupta/results'+str(cl),          # output directory
        num_train_epochs=10,              # total number of training epochs
        per_device_train_batch_size=4,  # batch size per device during training
        per_device_eval_batch_size=4,   # batch size for evaluation
        warmup_steps=500,               # number of warmup steps for learning rate scheduler
        weight_decay=0.01,              # strength of weight decay
        logging_dir='/ssd_scratch/cvit/avani.gupta/logs'+str(cl),            # directory for storing logs
        logging_steps=20000,
        save_steps = 30000
        )
        print("for class: ",mapping[cl])
        train_ind = np.where(train_labels==cl)[0].astype(int)
        new_train_labels = np.zeros(train_labels.shape).astype(int)
        new_train_labels[train_ind] = 1

        # val_ind = np.where(val_labels==cl)[0].astype(int)
        test_ind = np.where(test_labels==cl)[0].astype(int)
        new_test_labels = np.zeros(test_labels.shape).astype(int)
        new_test_labels[test_ind] = 1


        train_dataset = IMDbDataset(train_encodings, torch.from_numpy(new_train_labels))
        test_dataset = IMDbDataset(test_encodings, torch.from_numpy(new_test_labels))    


        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        model.to(device)
        #model = nn.DataParallel(model,device_ids=[0,1,2,3])

        trainer = Trainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            args=training_args,                  # training arguments, defined above
            train_dataset=train_dataset         # training dataset
        )
        trainer.train()
        # eval
        model.eval()
        torch.save(model, "bert_single_lb_classi"+str(cl)+".pth")
        val_dataloader = DataLoader(test_dataset, batch_size=8)
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        pred_lis = []
        gt_lis = []

        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            pred_lis.extend(predictions.cpu().numpy())
            gt_lis.extend(batch["labels"].cpu().numpy())

        calc_metrics(pred_lis, gt_lis)





PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


starting
for class:  aim


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/avani.gupta/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


...

metrics obtained in test: accuracy 0.75 recall 0.75, precision 0.75, f1-score 0.75
for class:  result


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/avani.gupta/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




metrics obtained in test: accuracy 0.8717948717948718 recall 0.8717948717948718, precision 0.8717948717948718, f1-score 0.8717948717948718


In [14]:
mapping

{0: 'aim', 1: 'hypothesis', 2: 'implication', 3: 'method', 4: 'result'}